In [ ]:
import warnings

warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss
from src.models import (
    ResNet34, EfficientNetV2, build_efficientnetv2,
    Var_ECAPA, MHA_LAS
)
from src.coatnet import CoAtNet

seed_everything(42, workers=True)

In [ ]:
num_classes = 5
dm = VoxCelebDataModule(batch_size=8)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
checkpoint_path = "checkpoints/resnet34/last.ckpt"
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams) \
        .load_from_checkpoint(
            checkpoint_path, 
            num_classes=num_classes, 
            loss_func=aams
        )
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint=checkpoint_path
)
trainer.test(
    model, 
    dm,
    ckpt_path=checkpoint_path
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=8)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    margin=0.2,
    scale=30
)
model = build_efficientnetv2(
    embeddings_dim=512,
    num_classes=5, 
    loss_func=subaams
)
logger = TensorBoardLogger("tb_logs", name="efficientnetv2")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=8)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    margin=0.2,
    scale=60
)
model = MHA_LAS(num_classes=5, loss_func=subaams)
logger = TensorBoardLogger("tb_logs", name="mha_las")
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=4)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = Var_ECAPA(num_classes=5, loss_func=aams)
logger = TensorBoardLogger("tb_logs", name="var_ecapa")
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
from torch import nn

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/coatnet/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=4)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = CoAtNet(in_ch=1, embeddings_dim=5, image_size=224, num_classes=5, loss_func=ce_loss)
logger = TensorBoardLogger("tb_logs", name="coatnet")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback]
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)